In [1]:
import requests
from datetime import datetime, timedelta
import pickle
import pandas as pd

In [2]:
key = '69ade1c55991714952e799be98757e7a-7813462b735c024c408b6f328d85ed24'

In [11]:
prices = {}
buckets = {}
date = datetime.strptime("2019-03-01T00:00:00Z", "%Y-%m-%dT%H:%M:%SZ").strftime("%Y-%m-%dT%H:%M:%SZ")
priceRange = None
df_list = []

while True:

    try:
        response = requests.get(
            url = 'https://api-fxpractice.oanda.com/v3/instruments/EUR_USD/orderBook',
            headers = {
                'Authorization' : 'Bearer ' + key,
            },
            data = {
                'time': date
            }
        ).json()
    except:
        print(date)

    date = (datetime.strptime(date, "%Y-%m-%dT%H:%M:%SZ") + timedelta(minutes=20)).strftime("%Y-%m-%dT%H:%M:%SZ")
    
    if 'orderBook' in response.keys():
        
        if not priceRange:
            priceRange = response['orderBook']['bucketWidth']
            price = response['orderBook']['price']
            numberToRound = len(str(response['orderBook']['bucketWidth']).split('.')[1])
            
            zerosCounter = 0
            for number in str(priceRange).split('.')[1]:
                if number == '0':
                    zerosCounter = zerosCounter + 1
                else:
                    break
                    
            middlePrice = round(float(price.split('.')[0] +\
                            '.' +\
                            price.split('.')[1][:3]) +\
                            float(priceRange), numberToRound)
        
        tmpBuckets = {}
        for bucket in response['orderBook']['buckets']:
            tmpBuckets[float(bucket['price'])] = {
                'l': bucket['longCountPercent'],
                's': bucket['shortCountPercent'],
            }
        pricesKeys = list(tmpBuckets.keys())
            
        row = [
            response['orderBook']['time'],
            response['orderBook']['price'],
            middlePrice,
        ]
        
        if middlePrice in pricesKeys:
            row.append(tmpBuckets[middlePrice]['l'])
            row.append(tmpBuckets[middlePrice]['s'])
        else:
            row.append(0)
            row.append(0)
        
        UpPrice = middlePrice
        DownPrice = middlePrice
        floatedPriceBucket = float(priceRange)
        for i in range(5):
            UpPrice = UpPrice + floatedPriceBucket
            if UpPrice in pricesKeys:
                row.append(tmpBuckets[UpPrice]['l'])
                row.append(tmpBuckets[UpPrice]['s'])
            else:
                row.append(0)
                row.append(0)
            
            DownPrice = DownPrice + floatedPriceBucket
            if DownPrice in pricesKeys:
                row.append(tmpBuckets[DownPrice]['l'])
                row.append(tmpBuckets[DownPrice]['s'])
            else:
                row.append(0)
                row.append(0)
        
        df_list.append(row)
    
    if date == '2019-03-12T00:00:00Z':
        break

In [12]:
df = pd.DataFrame(df_list, columns = [
    'time',
    'price',
    'roundedPrice',
    'level_0_l',
    'level_0_s',
    'level_1_up_l',
    'level_1_up_s',
    'level_1_down_l',
    'level_1_down_s',
    'level_2_up_l',
    'level_2_up_s',
    'level_2_down_l',
    'level_2_down_s',
    'level_3_up_l',
    'level_3_up_s',
    'level_3_down_l',
    'level_3_down_s',
    'level_4_up_l',
    'level_4_up_s',
    'level_4_down_l',
    'level_4_down_s',
    'level_5_up_l',
    'level_5_up_s',
    'level_5_down_l',
    'level_5_down_s',
])

In [13]:
df.tail(1)

,time,price,roundedPrice,level_0_l,level_0_s,level_1_up_l,level_1_up_s,level_1_down_l,level_1_down_s,level_2_up_l,...,level_3_down_l,level_3_down_s,level_4_up_l,level_4_up_s,level_4_down_l,level_4_down_s,level_5_up_l,level_5_up_s,level_5_down_l,level_5_down_s
791,2019-03-11T23:40:00Z,1.12563,1.1375,0.0646,0.1508,0.0892,0.0861,0.0892,0.0861,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
df.to_csv('march.csv')

In [55]:
tmp_jan = df

In [41]:
response['orderBook']['unixTime']

'1546300800'

In [7]:
response['orderBook'].keys()

dict_keys(['instrument', 'time', 'unixTime', 'price', 'bucketWidth', 'buckets'])

In [19]:
zerosCounter

3

1.1465

In [5]:
prices

{'2019-01-01T00:00:00Z': '1.14649'}

In [4]:
buckets

{'2019-01-01T00:00:00Z': [{'price': '0.00000',
   'longCountPercent': '0.0253',
   'shortCountPercent': '0.1140'},
  {'price': '0.00050',
   'longCountPercent': '0.0000',
   'shortCountPercent': '0.0063'},
  {'price': '0.00100',
   'longCountPercent': '0.0095',
   'shortCountPercent': '0.0032'},
  {'price': '0.00500',
   'longCountPercent': '0.0000',
   'shortCountPercent': '0.0032'},
  {'price': '0.01000',
   'longCountPercent': '0.0063',
   'shortCountPercent': '0.0063'},
  {'price': '0.01200',
   'longCountPercent': '0.1330',
   'shortCountPercent': '0.0000'},
  {'price': '0.01750',
   'longCountPercent': '0.0000',
   'shortCountPercent': '0.0032'},
  {'price': '0.02000',
   'longCountPercent': '0.0032',
   'shortCountPercent': '0.0032'},
  {'price': '0.02300',
   'longCountPercent': '0.0000',
   'shortCountPercent': '0.0032'},
  {'price': '0.02700',
   'longCountPercent': '0.0000',
   'shortCountPercent': '0.0032'},
  {'price': '0.04350',
   'longCountPercent': '0.0000',
   'shortC

In [ ]:
len(prices.keys())

In [ ]:
prices_january = prices
buckets_january = buckets

In [10]:
with open('buckets_january', 'wb') as f:
    pickle.dump(buckets_january, f)

In [11]:
with open('prices_january', 'wb') as f:
    pickle.dump(prices_january, f)

In [17]:
with open('buckets_january', 'rb') as f:
    pickle.load(f)

EOFError: Ran out of input

In [87]:
prices_february = prices

In [76]:
prices_march = prices

In [95]:
merged_prices = {**prices_january, **prices_february, **prices_march}

In [61]:
response

{'orderBook': {'instrument': 'EUR_USD',
  'time': '2019-03-05T17:00:00Z',
  'unixTime': '1551805200',
  'price': '1.12988',
  'bucketWidth': '0.00050',
  'buckets': [{'price': '0.00000',
    'longCountPercent': '0.0300',
    'shortCountPercent': '0.0599'},
   {'price': '0.00050',
    'longCountPercent': '0.0000',
    'shortCountPercent': '0.0027'},
   {'price': '0.00100',
    'longCountPercent': '0.0082',
    'shortCountPercent': '0.0054'},
   {'price': '0.00300',
    'longCountPercent': '0.0027',
    'shortCountPercent': '0.0000'},
   {'price': '0.00500',
    'longCountPercent': '0.0027',
    'shortCountPercent': '0.0054'},
   {'price': '0.00550',
    'longCountPercent': '0.0027',
    'shortCountPercent': '0.0000'},
   {'price': '0.00800',
    'longCountPercent': '0.0027',
    'shortCountPercent': '0.0000'},
   {'price': '0.01000',
    'longCountPercent': '0.0082',
    'shortCountPercent': '0.0082'},
   {'price': '0.01200',
    'longCountPercent': '0.1144',
    'shortCountPercent': '0

In [53]:
response

{'instrument': 'EUR_USD',
 'time': '2019-01-13T04:20:00Z',
 'unixTime': '1547353200',
 'price': '1.14687',
 'bucketWidth': '0.00050',
 'buckets': [{'price': '0.00000',
   'longCountPercent': '0.0265',
   'shortCountPercent': '0.0630'},
  {'price': '0.00050',
   'longCountPercent': '0.0000',
   'shortCountPercent': '0.0033'},
  {'price': '0.00100',
   'longCountPercent': '0.0099',
   'shortCountPercent': '0.0033'},
  {'price': '0.00300',
   'longCountPercent': '0.0033',
   'shortCountPercent': '0.0000'},
  {'price': '0.00500',
   'longCountPercent': '0.0000',
   'shortCountPercent': '0.0033'},
  {'price': '0.01000',
   'longCountPercent': '0.0066',
   'shortCountPercent': '0.0066'},
  {'price': '0.01200',
   'longCountPercent': '0.1393',
   'shortCountPercent': '0.0000'},
  {'price': '0.01750',
   'longCountPercent': '0.0000',
   'shortCountPercent': '0.0033'},
  {'price': '0.02000',
   'longCountPercent': '0.0066',
   'shortCountPercent': '0.0033'},
  {'price': '0.02300',
   'longCount

In [ ]:
response

In [4]:
l_sum = 0
s_sum = 0

for bucket in response['buckets']:
    l_sum = l_sum + float(bucket['longCountPercent'])
    s_sum = s_sum + float(bucket['shortCountPercent'])

In [5]:
l_sum

44.24530000000091

In [6]:
s_sum

55.93520000000011

In [7]:
l_sum + s_sum

100.18050000000102

In [8]:
bucket

{'price': '7000000000.00000',
 'longCountPercent': '0.0000',
 'shortCountPercent': '0.0028'}